# 1차 모델 학습하기

본 실험의 1차 모델은 범주형 변수의 의미적 관계를 고려하지 않은 임시 수치화(Label Encoding) 상태로 학습되었으며    
이는 모델 성능 자체보다 전처리 및 피처 엔지니어링 효과를 비교하기 위한 기준선(Baseline) 으로 사용하였다.

1차 모델은 LightGBM으로 설정하였다.    
Tree 기반 모델인 LightGBM은 범주형 변수를 category dtype으로 지정할 경우 One-Hot Encoding 없이도 분기 기준을 학습할 수 있다.    
이에 본 실험에서는 EDA 이전 단계에서도 최소한의 전처리만 적용한 모델을 구성하여 피처 엔지니어링 효과를 비교하였다.

In [1]:
# 라이브러리
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
# 데이터 불러오기
final_test = pd.read_csv("./data/final_Feature/test_final.csv")
final_train = pd.read_csv("./data/final_Feature/train_final.csv")

# 1. 모델이 학습할 수 있도록 데이터 준비하기

## 모델 학습에 제외 될 컬럼
- AccidentId : 식별자로 예측 신호가 없음
- Gravity : 타깃
- PostalAddress / GPSCode : : 주소 및 GPS 코드로, 범주 수가 많고 노이즈로 작용할 가능성이 높음

In [3]:
# df_train에 학습용 변수 저장하기
df_train = final_train.drop(columns=["AccidentId", "Gravity", "PostalAddress", "GPSCode"])

In [4]:
# 저장된 df_train 확인
df_train.head(3)

,Date,Hour,Light,Department,Commune,InAgglomeration,IntersectionType,Weather,CollisionType,Latitude,...,Maneuver_PassRight,Maneuver_Reverse,Maneuver_SameDirectionOrLane,Maneuver_Stopped,Maneuver_SwerveToLeft,Maneuver_SwerveToRight,Maneuver_TurnToLeft,Maneuver_TurnToRight,Maneuver_UTurnInLane,Maneuver_WrongWay
0,2018-01-24,0 days 15:05:00,Daylight,590,5,No,Y-type,Normal,2Vehicles-BehindVehicles-Frontal,50.555225,...,0,0,0,0,0,0,1,0,0,0
1,2018-02-12,0 days 10:15:00,Daylight,590,11,Yes,Square,VeryGood,NoCollision,50.529369,...,0,0,0,0,0,0,0,0,0,0
2,2018-04-03,0 days 11:35:00,Daylight,590,477,Yes,T-type,Normal,NoCollision,50.510923,...,0,0,0,0,0,0,1,0,0,0


In [5]:
# dtype 확인하기
df_train.dtypes

Date                      object
Hour                      object
Light                     object
Department                 int64
Commune                    int64
                           ...  
Maneuver_SwerveToRight     int64
Maneuver_TurnToLeft        int64
Maneuver_TurnToRight       int64
Maneuver_UTurnInLane       int64
Maneuver_WrongWay          int64
Length: 137, dtype: object

## Date
- 월·일 정보만 추출하여 MMDD 형태로 변환
- MMDD 값을 범주형(category) 변수로 사용
- 날짜를 단순 숫자형으로 입력할 경우 모델이 값의 크기 기준으로 분기하여 실제 시간적 의미와 다른 방식으로 학습할 가능성이 있다.
- 이를 방지하기 위해 월·일을 숫자 형태로 표현하되 category dtype으로 지정하여 순서나 크기 의미 없이 범주형 변수로 학습하도록 처리한다.

In [6]:
# datetime으로 변환 -> 월일 추출 -> category dtype 변경하기
df_train["Date"] = ( pd.to_datetime(df_train["Date"], errors="coerce").dt.strftime("%m%d").astype("category"))

## Hour
- 시간만 추출하여 HH 형태로 변환
- HH 값을 int 변수로 사용
- 시간도 날짜와 동일하게 숫자형으로 입력하면 실제 시간적 의미와 다른 방식으로 학습될 가능성이 있다.
- 이를 방지하기 위해 시간만 추출하되 int64 dtype으로 지정하여 학습하도록 처리한다.

In [7]:
# 시간만 추출하기 dtype = int64
df_train["Hour"] = (pd.to_timedelta(df_train["Hour"]).dt.components["hours"].astype("int64"))

## 범주형 변수 Category type로 변경
범주형 변수는 문자열(object) 형태로는 모델 입력으로 사용할 수 없기 때문에    
LightGBM의 categorical feature 처리 방식을 활용할 수 있도록 category dtype으로 변환하여 학습에 사용하였다.

In [8]:
# object 변수 선택
cols = [
    "Light", "InAgglomeration", "IntersectionType", "Weather",
    "CollisionType", "RoadType", "RoadLetter", "Circulation", "SpecialLane",
    "Slope", "Layout", "SurfaceCondition", "Infrastructure", "Localization"
]

In [9]:
# category 타입으로 변환
df_train[cols] = df_train[cols].astype("category")

In [10]:
# 최종 dtype 확인하기
pd.set_option("display.max_rows", None)
df_train.dtypes

Date                                            category
Hour                                               int64
Light                                           category
Department                                         int64
Commune                                            int64
InAgglomeration                                 category
IntersectionType                                category
Weather                                         category
CollisionType                                   category
Latitude                                         float64
Longitude                                        float64
RoadType                                        category
RoadNumber                                         int64
RoadSecNumber                                    float64
RoadLetter                                      category
Circulation                                     category
LaneNumber                                         int64
SpecialLane                    

# 2. 데이터 분할하기

In [11]:
# Y-train 만들기
# Y_train은 타깃으로 final_train의 [Gravity]를 사용한다.

y_train = final_train['Gravity']

In [12]:
y_train.head(5)

0    NonLethal
1    NonLethal
2    NonLethal
3    NonLethal
4       Lethal
Name: Gravity, dtype: object

In [13]:
# 변수 분할

X = df_train
y = y_train

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 3. LightGBM 학습

In [14]:
# 학습하기

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model = LGBMClassifier(
    n_estimators=500,      
    learning_rate=0.05,    
    random_state=42
)

model.fit(X_tr, y_tr)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 36126, number of negative: 2126
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010010 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3439
[LightGBM] [Info] Number of data points in the train set: 38252, number of used features: 136
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.944421 -> initscore=2.832771
[LightGBM] [Info] Start training from score 2.832771


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [15]:
# 예측 확률 확인
y_proba = model.predict_proba(X_val)[:, 1]

y_proba

array([0.98437103, 0.99845364, 0.99170469, ..., 0.84025174, 0.99417429,
       0.97891734], shape=(9564,))

In [16]:
# 이진 예측
y_pred = model.predict(X_val)

y_pred

array(['NonLethal', 'NonLethal', 'NonLethal', ..., 'NonLethal',
       'NonLethal', 'NonLethal'], shape=(9564,), dtype=object)

In [17]:
# AUC 확인
print("AUC:", roc_auc_score(y_val, y_proba))

AUC: 0.783177149583436


In [18]:
# F1_score 확인
print("F1:", f1_score(y_val, y_pred, pos_label="Lethal"))

F1: 0.0773109243697479


In [19]:
# confusion matrix (혼동행렬 확인)
print("confusion matrix : ")
print(confusion_matrix(y_val, y_pred, labels=["NonLethal", "Lethal"]))

confusion matrix : 
[[8992   40]
 [ 509   23]]


In [20]:
# Classification Report
print("Classification Report")
print(classification_report(y_val, y_pred, labels=["NonLethal", "Lethal"]))

Classification Report
              precision    recall  f1-score   support

   NonLethal       0.95      1.00      0.97      9032
      Lethal       0.37      0.04      0.08       532

    accuracy                           0.94      9564
   macro avg       0.66      0.52      0.52      9564
weighted avg       0.91      0.94      0.92      9564



# 결론

본 문제에서는 치명 사고를 높치는 FN이 가장 치명적인 오류이므로 EDA 및 피처 엔지니어링을 통해 이를 보완한 후 다시 모델 학습을 진할 예정이다.    
현재 모델은 심각하지 않은 사고를 심각하다고 판단하는 FP의 수가 이미 낮은 편이므로 심각한 사고를 더 많이 탐지하여 FN을 감소시키는 것을 주요 목표로 설정한다.    
이 과정에서 FP가 일부 증가하더라도 F1-score와 치명 사고 재현율(recall) 개선을 우선적으로 고려하는 전략을 적용할 예정이다.

FN / TP 를 따로 csv 파일로 저장하여 다음 단계에서 분석하여 Feature_Engieering 할 예정이다.

In [21]:
# FN / TP 집단 만들어 확인하기

df_val = X_val.copy()
df_val["y_true"] = y_val
df_val["y_pred"] = y_pred

In [22]:
# FN 집단
fn_df = df_val[(df_val["y_true"] == "Lethal") & (df_val["y_pred"] == "NonLethal")]

# TP 집
tp_df = df_val[(df_val["y_true"] == "Lethal") & (df_val["y_pred"] == "Lethal")]

In [23]:
# dtype 유지해서 저장하기
fn_df.to_pickle("./data/fn_tp_pkl/fn_cases.pkl")
tp_df.to_pickle("./data/fn_tp_pkl/tp_cases.pkl")